## Entremes

#### Transacciones PDF 

In [77]:
pip install pymupdf pdfplumber tabula-py camelot-py[cv] pytesseract Pillow pandas camelot

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [78]:
# Lectura de pdf
import fitz  
import pdfplumber
import tabula
import pytesseract
from PIL import Image
import pandas as pd
import io
import re
import pandas as pd
import unicodedata
from datetime import datetime, timedelta


In [79]:


class PDFExtractorAdaptativo:
    def __init__(self):
        self.metodos_texto = ['pdfplumber', 'pymupdf', 'ocr']
        self.metodos_tablas = ['tabula', 'pdfplumber']  # Quitamos 'camelot'
    
    def detectar_tipo_contenido(self, archivo_pdf):
        """Detecta qué tipo de contenido tiene el PDF"""
        try:
            doc = fitz.open(archivo_pdf)
            primera_pagina = doc[0]
            
            texto = primera_pagina.get_text().strip()
            imagenes = primera_pagina.get_images()
            
            with pdfplumber.open(archivo_pdf) as pdf:
                page = pdf.pages[0]
                tablas_detectadas = page.find_tables()
            
            resultado = {
                'tiene_texto': len(texto) > 50,
                'tiene_imagenes': len(imagenes) > 0,
                'posibles_tablas': len(tablas_detectadas) > 0,
                'es_escaneado': len(texto) < 50 and len(imagenes) > 0,
                'calidad_texto': 'alta' if len(texto) > 200 else 'baja'
            }
            
            doc.close()
            return resultado
            
        except Exception as e:
            return {'error': str(e)}
    
    def extraer_texto_robusto(self, archivo_pdf):
        """Extrae texto probando diferentes métodos"""
        metodos_resultados = {}
        
        # pdfplumber
        try:
            with pdfplumber.open(archivo_pdf) as pdf:
                texto = ""
                for page in pdf.pages:
                    texto += page.extract_text() + "\n"
                metodos_resultados['pdfplumber'] = texto
        except Exception as e:
            metodos_resultados['pdfplumber'] = f"Error: {e}"
        
        # pymupdf
        try:
            doc = fitz.open(archivo_pdf)
            texto = ""
            for page in doc:
                texto += page.get_text() + "\n"
            doc.close()
            metodos_resultados['pymupdf'] = texto
        except Exception as e:
            metodos_resultados['pymupdf'] = f"Error: {e}"
        
        # OCR
        try:
            doc = fitz.open(archivo_pdf)
            texto = ""
            for page_num in range(len(doc)):
                page = doc.load_page(page_num)
                pix = page.get_pixmap()
                img_data = pix.tobytes("png")
                img = Image.open(io.BytesIO(img_data))
                texto += pytesseract.image_to_string(img, lang='spa') + "\n"
            doc.close()
            metodos_resultados['ocr'] = texto
        except Exception as e:
            metodos_resultados['ocr'] = f"Error: {e}"
        
        # Seleccionar mejor resultado
        mejor_resultado = ""
        max_longitud = 0
        for metodo, resultado in metodos_resultados.items():
            if not resultado.startswith("Error") and len(resultado) > max_longitud:
                mejor_resultado = resultado
                max_longitud = len(resultado)
        
        return {
            'texto_final': mejor_resultado,
            'metodos_intentados': metodos_resultados,
            'metodo_exitoso': max([k for k, v in metodos_resultados.items() 
                                 if not v.startswith("Error") and len(v) == max_longitud], 
                                default="ninguno")
        }
    
    def extraer_tablas_robusto(self, archivo_pdf):
        """Extrae tablas probando diferentes métodos (sin camelot)"""
        resultados_tablas = {}
        
        # tabula-py
        try:
            tablas = tabula.read_pdf(archivo_pdf, pages='all', multiple_tables=True)
            resultados_tablas['tabula'] = {
                'tablas': tablas,
                'cantidad': len(tablas),
                'exito': True
            }
        except Exception as e:
            resultados_tablas['tabula'] = {'error': str(e), 'exito': False}
        
        # pdfplumber
        try:
            with pdfplumber.open(archivo_pdf) as pdf:
                todas_tablas = []
                for page in pdf.pages:
                    tablas_pagina = page.extract_tables()
                    for tabla in tablas_pagina:
                        df = pd.DataFrame(tabla[1:], columns=tabla[0])
                        todas_tablas.append(df)
                
                resultados_tablas['pdfplumber'] = {
                    'tablas': todas_tablas,
                    'cantidad': len(todas_tablas),
                    'exito': True
                }
        except Exception as e:
            resultados_tablas['pdfplumber'] = {'error': str(e), 'exito': False}
        
        return resultados_tablas
    
    def procesar_pdf_completo(self, archivo_pdf):
        """Proceso completo adaptativo"""
        print(f"Analizando: {archivo_pdf}")
        
        info_contenido = self.detectar_tipo_contenido(archivo_pdf)
        print(f"Análisis de contenido: {info_contenido}")
        
        resultado_final = {
            'archivo': archivo_pdf,
            'analisis_contenido': info_contenido,
            'texto': None,
            'tablas': None
        }
        
        if info_contenido.get('tiene_texto') or info_contenido.get('es_escaneado'):
            print("Extrayendo texto...")
            resultado_texto = self.extraer_texto_robusto(archivo_pdf)
            resultado_final['texto'] = resultado_texto
        
        if info_contenido.get('posibles_tablas'):
            print("Extrayendo tablas...")
            resultado_tablas = self.extraer_tablas_robusto(archivo_pdf)
            resultado_final['tablas'] = resultado_tablas
        
        return resultado_final

# Ejemplo de uso
if __name__ == "__main__":
    extractor = PDFExtractorAdaptativo()
    resultado = extractor.procesar_pdf_completo("resources/ENTREMES.pdf")
    print("Resultado final:")
    print(resultado)


Analizando: resources/ENTREMES.pdf


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


Análisis de contenido: {'tiene_texto': True, 'tiene_imagenes': True, 'posibles_tablas': True, 'es_escaneado': False, 'calidad_texto': 'alta'}
Extrayendo texto...


CropBox missing from /Page, defaulting to MediaBox


Extrayendo tablas...


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


Resultado final:
{'archivo': 'resources/ENTREMES.pdf', 'analisis_contenido': {'tiene_texto': True, 'tiene_imagenes': True, 'posibles_tablas': True, 'es_escaneado': False, 'calidad_texto': 'alta'}, 'texto': {'texto_final': 'Dirección de Envío\nEntremés Máquinas Dispensadoras SAS\nCalle 79b # 29-15, Barrios Unidos\nBOGOTÁ, D.C. Bogotá 111511\nColombia\n\uf095 +57 312 7086927\nCROC S.A.S.\nCL 69 19 36\nBogotá D.C Bogotá\nColombia\n\uf095 3 4 7 4 5 2 9\nNIT: 830125610-1\nPedido de compra #P03260\nRepresentante de Compra:\nAbastecimiento de Entremés\nFecha de la Orden:\n26-05-2025 11:33:20\nDescripción\nCtdad\nPrecio Unitario\nImporte\nChicharron Don Chicharron Espiral BBQ x 21Gr\n120,00 Unidades\n1.868,00\n$ 224.160,00\nChicharron Don Chicharron Espiral Natural x 21Gr\n120,00 Unidades\n1.868,00\n$ 224.160,00\nMonto Libre de\nImpuestos\n$ 448.320,00\nIVA 5%\n$ 22.416,00\nTotal\n$ 470.736,00\nEntremés Máquinas Dispensadoras\nSAS\nCalle 79b # 29-15, Barrios Unidos\nBogotá 111211\n312 7086927 

In [80]:
#funcion de limpieza de columnas : 

def limpiar_columnas(df):
    # Convertimos los nombres de columna a string
    df.columns = df.columns.astype(str)

    # Normalizamos las tildes usando unicodedata
    df.columns = [
        unicodedata.normalize('NFKD', col)
        .encode('ascii', 'ignore')
        .decode('utf-8')
        for col in df.columns
    ]

    # Aplicamos las transformaciones
    df.columns = (
        pd.Index(df.columns)
        .str.strip()
        .str.lower()
        .str.replace(' ', '_')
        .str.replace(r'[^\w\s]', '', regex=True)
    )
    
    return df


In [81]:
#extraer del diccionario final tablas y texto
textos = resultado.get('texto', {}).get('texto_final', '')
print(type(textos))

#Escritura .txt
with open("ENTREMES.txt", "w", encoding="utf-8") as f:
    f.write(textos)

<class 'str'>


In [82]:
datos_generales = {
    "Dirección Envío Empresa 1": re.search(r"Dirección de Envío\s*\n(.+)", textos).group(1),
    "Empresa 1": re.search(r"Dirección de Envío\s*\n.+\n(.+)", textos).group(1),
    "Dirección Empresa 1": re.search(r"Dirección de Envío[\s\S]+?\n(.+)\n(.+)\nColombia", textos).group(1),
    "Ciudad Empresa 1": re.search(r"Dirección de Envío[\s\S]+?\n.+\n(.+)", textos).group(1),
    "Teléfono Empresa 1": re.search(r"Dirección de Envío[\s\S]+?\n.+\n.+\nColombia\n.*?(\+?\d[\d\s]+)", textos).group(1).replace(" ", ""),
    
    "Empresa 2": re.search(r"\+?\d[\d\s]+\n(.+)", textos).group(1),
    "Dirección Empresa 2": re.search(r"\+?\d[\d\s]+\n.+\n(.+)", textos).group(1),
    "Ciudad Empresa 2": re.search(r"\+?\d[\d\s]+\n.+\n.+\n(.+)", textos).group(1),
    "Teléfono Empresa 2": re.search(r"\+?\d[\d\s]+\n.+\n.+\n.+\nColombia\n\s*([\d\s]+)", textos).group(1).replace(" ", ""),
    
    "NIT": re.search(r"NIT:\s*(\d+-\d+)", textos).group(1),
    "Pedido de Compra": re.search(r"Pedido de compra\s*#(\S+)", textos).group(1),
    "Representante de Compra": re.search(r"Representante de Compra:\s*(.+)", textos).group(1),
    "Fecha de Orden": re.search(r"Fecha de la Orden:\s*(.+)", textos).group(1),

    "Monto Libre de Impuestos": re.search(r"Monto Libre de\s*Impuestos\s*\$ ([\d.,]+)", textos).group(1),
    "IVA 5%": re.search(r"IVA 5%\s*\$ ([\d.,]+)", textos).group(1),
    "Total": re.search(r"Total\s*\$ ([\d.,]+)", textos).group(1),
    
    "Correo Empresa 1": re.search(r"(\S+@\S+)", textos).group(1),
    "Sitio Web Empresa 1": re.search(r"(https?://\S+)", textos).group(1),
    "Página": re.search(r"Página:\s*(.+)", textos).group(1),
}


In [83]:
df_datos_generales = pd.DataFrame([datos_generales])
df_datos_generales  = limpiar_columnas(df_datos_generales )

In [84]:
df_datos_generales

,direccion_envio_empresa_1,empresa_1,direccion_empresa_1,ciudad_empresa_1,telefono_empresa_1,empresa_2,direccion_empresa_2,ciudad_empresa_2,telefono_empresa_2,nit,pedido_de_compra,representante_de_compra,fecha_de_orden,monto_libre_de_impuestos,iva_5,total,correo_empresa_1,sitio_web_empresa_1,pagina
0,Entremés Máquinas Dispensadoras SAS,"Calle 79b # 29-15, Barrios Unidos","Calle 79b # 29-15, Barrios Unidos","BOGOTÁ, D.C. Bogotá 111511",+573127086927\n,Colombia, +57 312 7086927,CROC S.A.S.,3474529\n,830125610-1,P03260,Abastecimiento de Entremés,26-05-2025 11:33:20,"448.320,00","22.416,00","470.736,00",info@entremesvm.com,https://entremesvm.com,1 de 1


### Escoger Columnas -Ordenar dataframe
1. Cliente
2. Direccion
3. Destino
4. Numero de Pedido
5. Fecha de Orden
6. Fecha de Entrega
7. Id Falso (Para Cruce de Orden de compra )

In [85]:
#Seleccion de columnas 
df_entremes = df_datos_generales[[
    "direccion_envio_empresa_1",
    "empresa_1",
    "direccion_empresa_1",
    "pedido_de_compra",
    "fecha_de_orden"
   
]]

In [86]:
#Rename Columns 
df_entremes.rename(columns={
            'direccion_envio_empresa_1': 'CLIENTE',
            'empresa_1': 'DIRECCION',
            'direccion_empresa_1': 'DESTINO',
            'pedido_de_compra': 'NUMERO_DE_PEDIDO',
            'fecha_de_orden': 'FECHA_DE_ORDEN'
}, inplace=True)

# Create a false column for id
df_entremes['id'] = 1

#Dates 
df_entremes['FECHA_DE_ENTREGA'] = pd.to_datetime(df_entremes['FECHA_DE_ORDEN'], format="%d-%m-%Y %H:%M:%S")
df_entremes['FECHA_DE_ORDEN'] = pd.to_datetime(df_entremes['FECHA_DE_ORDEN'], format="%d-%m-%Y %H:%M:%S")
df_entremes['FECHA_DE_ENTREGA'] = df_entremes['FECHA_DE_ENTREGA'] + timedelta(days=8)
df_entremes['FECHA_DE_ENTREGA'] = df_entremes['FECHA_DE_ENTREGA'].dt.date
df_entremes['FECHA_DE_ORDEN'] = df_entremes['FECHA_DE_ORDEN'].dt.date

C:\Users\Analista de  Datos\AppData\Local\Temp\ipykernel_13964\56697425.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_entremes.rename(columns={
C:\Users\Analista de  Datos\AppData\Local\Temp\ipykernel_13964\56697425.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_entremes['id'] = 1
C:\Users\Analista de  Datos\AppData\Local\Temp\ipykernel_13964\56697425.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

In [87]:
df_entremes

,CLIENTE,DIRECCION,DESTINO,NUMERO_DE_PEDIDO,FECHA_DE_ORDEN,id,FECHA_DE_ENTREGA
0,Entremés Máquinas Dispensadoras SAS,"Calle 79b # 29-15, Barrios Unidos","Calle 79b # 29-15, Barrios Unidos",P03260,2025-05-26,1,2025-06-03


#### Escoger Columnas Productos 
1. Numero de item
2. descripcion
3. Cantidad
4. Unidad de medida
5. Costo Unitario
6. importe
7. id falso (Unir para realizar insersion de datos)

In [88]:
# ------------------------------
# EXTRACCIÓN DE LA TABLA DE PRODUCTOS
# ------------------------------
tabla_bruta = textos.split(" Descripción")[-1]
lineas = tabla_bruta.strip().splitlines()
lineas = [line.strip() for line in lineas if line.strip() != ""]

In [89]:
lineas = textos.splitlines()

# 1. Buscamos la posición de la cabecera de productos
try:
    inicio = lineas.index("Descripción")
except ValueError:
    raise RuntimeError("No encontré la sección de productos")

# 2. Inicializamos la lista y el índice justo después de "Descripción"
productos = []
i = inicio + 1

# 3. Recorremos hasta toparnos con "Monto Libre de"
while i < len(lineas) and not lineas[i].startswith("Monto Libre de"):
    # Nos aseguramos de que haya al menos 4 líneas disponibles
    if i + 3 < len(lineas):
        desc = lineas[i].strip()

        # Línea con cantidad y unidad (por ejemplo: "120,00 Unidades")
        cant_unid = lineas[i+1].strip()
        m = re.match(r"([\d.,]+)\s+(\w+)", cant_unid)
        if not m:
            i += 1
            continue
        cantidad, unidad = m.groups()

        # Precio unitario (por ejemplo: "1.868,00")
        precio_unit = lineas[i+2].strip().replace(".", "").replace(",", ".")
        
        # Importe (por ejemplo: "$ 224.160,00")
        importe = lineas[i+3].strip()
        # quitamos símbolo $, espacios y formateamos a punto decimal
        importe = importe.replace("$", "").replace(" ", "").replace(".", "").replace(",", ".")

        productos.append({
            "Descripción": desc,
            "Cantidad": cantidad,
            "Unidad de Medida": unidad,
            "Precio Unitario": precio_unit,
            "Importe Total": importe
        })
        # avanzamos 4 líneas para el siguiente producto
        i += 4
    else:
        break

# 4. Creamos el DataFrame
df_productos = pd.DataFrame(productos)

# Si quieres convertir cadenas a números:
df_productos["Cantidad"] = df_productos["Cantidad"].str.replace(",", ".").astype(float)
df_productos["Precio Unitario"] = df_productos["Precio Unitario"].astype(float)
df_productos["Importe Total"] = df_productos["Importe Total"].astype(float)


In [90]:
df_productos = limpiar_columnas(df_productos)

In [91]:
#Codigo Incremental Item

df_productos['item'] = range(1, len(df_productos) + 1)

# Renombrar Columnas 
df_productos.rename(columns={
    'item': 'NUMERO_DE_ITEM',
    'descripcion' :'DESCRIPCION',
    'cantidad': 'CANTIDAD',
    'unidad_de_medida': 'UNIDAD_DE_MEDIDA',
    'precio_unitario': 'COSTO_UNITARIO',
    'importe_total': 'IMPORTE'
}, inplace=True)

# Añadir columna de ID
df_productos['id'] = 1

In [92]:
df_productos

,DESCRIPCION,CANTIDAD,UNIDAD_DE_MEDIDA,COSTO_UNITARIO,IMPORTE,NUMERO_DE_ITEM,id
0,Chicharron Don Chicharron Espiral BBQ x 21Gr,120.0,Unidades,1868.0,224160.0,1,1
1,Chicharron Don Chicharron Espiral Natural x 21Gr,120.0,Unidades,1868.0,224160.0,2,1


#### Join 

In [93]:
df_join  = df_entremes.merge(df_productos, on='id', how='inner')

# Drop Duplicates 
df_join  = df_join.drop_duplicates()

# Reordenar columnas
df_join = df_join[['id', 'CLIENTE', 'DIRECCION', 'DESTINO', 'NUMERO_DE_PEDIDO', 'FECHA_DE_ORDEN', 'FECHA_DE_ENTREGA',
                   'NUMERO_DE_ITEM', 'DESCRIPCION', 'CANTIDAD', 'UNIDAD_DE_MEDIDA', 'COSTO_UNITARIO', 'IMPORTE']]

In [94]:
df_join 

,id,CLIENTE,DIRECCION,DESTINO,NUMERO_DE_PEDIDO,FECHA_DE_ORDEN,FECHA_DE_ENTREGA,NUMERO_DE_ITEM,DESCRIPCION,CANTIDAD,UNIDAD_DE_MEDIDA,COSTO_UNITARIO,IMPORTE
0,1,Entremés Máquinas Dispensadoras SAS,"Calle 79b # 29-15, Barrios Unidos","Calle 79b # 29-15, Barrios Unidos",P03260,2025-05-26,2025-06-03,1,Chicharron Don Chicharron Espiral BBQ x 21Gr,120.0,Unidades,1868.0,224160.0
1,1,Entremés Máquinas Dispensadoras SAS,"Calle 79b # 29-15, Barrios Unidos","Calle 79b # 29-15, Barrios Unidos",P03260,2025-05-26,2025-06-03,2,Chicharron Don Chicharron Espiral Natural x 21Gr,120.0,Unidades,1868.0,224160.0


In [ ]:
#Destino por Bogota , Buscar gramaje, descripcion para traer el producto correcto

